In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU, Conv1D, MaxPooling1D, GlobalMaxPooling1D,Flatten
from sklearn.metrics import classification_report, accuracy_score


### **Load and Preprocess Data**

In [3]:
df = pd.read_csv('Reddit_Encoded.csv')

In [4]:
df.head()

,comment,hate_speech,lemmatized_comment,document_vector_flat
0,subsection retarded hungarians ohh boy brace l...,1,subsection retard hungarians ohh boy brace liv...,"0.014043219,-0.01809359,0.017145459,0.08062436..."
1,hiii just got work Foundation and grounding ma...,0,hiii just get work Foundation and ground mainl...,"-0.0030388932,-0.035133556,0.020659983,0.07383..."
2,wow guess soyboys every country,0,wow guess soyboys every country,"0.017362628,0.005587179,0.0297773,0.109146975,..."
3,owen benjamins soyboy song goes every country ...,0,owen benjamins soyboy song go every country amaze,"0.018085241,0.0011954829,2.8959475e-05,0.07601..."
4,yall hear sumn means live small town rn for w...,0,yall hear sumn mean live small town rn for wor...,"0.023993038,-0.00060867134,0.005239945,0.05989..."


In [10]:
# Convert the document vectors from string to numerical form
df['document_vector'] = df['document_vector_flat'].apply(lambda x: np.fromstring(x, sep=','))

# Extract features and labels
X = np.array(df['document_vector'].tolist())
y = df['hate_speech'].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN (add a channel dimension)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## ***Deep Learning Models***

**CNN Model**

In [17]:
# Define the model
model = Sequential()

# Add a Conv1D layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))

# Add a MaxPooling layer
model.add(MaxPooling1D(pool_size=2))

# Add a Flatten layer to convert 3D data to 1D
model.add(Flatten())

# Add a Dense layer
model.add(Dense(128, activation='relu'))

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Add the output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


C:\Users\Aman Shekhar Sachan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 298, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 149, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 9536)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       1,220,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,221,121 (4.66 MB)

 Trainable params: 1,221,121 (4.66 MB)

 Non-trainable params: 0 (0.00 B)

### **Train and Evaluate Model**

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on validation data
score = model.evaluate(X_val, y_val, verbose=0)
print(f"CNN Validation Accuracy: {score[1]}")

Epoch 1/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 27s 48ms/step - accuracy: 0.8710 - loss: 0.3178 - val_accuracy: 0.8557 - val_loss: 0.3730
Epoch 2/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.8692 - loss: 0.3174 - val_accuracy: 0.8515 - val_loss: 0.3758
Epoch 3/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 42s 49ms/step - accuracy: 0.8703 - loss: 0.3151 - val_accuracy: 0.8562 - val_loss: 0.3699
Epoch 4/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 47s 59ms/step - accuracy: 0.8764 - loss: 0.3035 - val_accuracy: 0.8530 - val_loss: 0.3880
Epoch 5/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 37s 51ms/step - accuracy: 0.8727 - loss: 0.3173 - val_accuracy: 0.8605 - val_loss: 0.3721
Epoch 6/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 38s 45ms/step - accuracy: 0.8770 - loss: 0.2973 - val_accuracy: 0.8562 - val_loss: 0.3947
Epoch 7/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.8689 - loss: 0.3109 - val_accuracy: 0.8609 - val_loss: 0.3811
Epoch 8/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8823 - loss: 0.2886 - 

In [16]:
# Predict the labels for the validation data
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int).flatten()

# Generate and print the classification report
report = classification_report(y_val, y_pred, target_names=['Not Hate Speech', 'Hate Speech'])
print(report)

139/139 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step
                 precision    recall  f1-score   support

Not Hate Speech       0.90      0.92      0.91      3369
    Hate Speech       0.72      0.66      0.69      1074

       accuracy                           0.86      4443
      macro avg       0.81      0.79      0.80      4443
   weighted avg       0.85      0.86      0.86      4443



**LSTM**

In [19]:
# Define the LSTM model
model = Sequential()

# Add an LSTM layer
model.add(LSTM(units=128, input_shape=(X_train.shape[1], 1)))

# Add a Dense layer with 128 units
model.add(Dense(128, activation='relu'))

# Add a Dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Add the output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


C:\Users\Aman Shekhar Sachan\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 83,201 (325.00 KB)

 Trainable params: 83,201 (325.00 KB)

 Non-trainable params: 0 (0.00 B)

### **Train and Evaluate Model**

In [20]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on validation data
score = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {score[1]}")


Epoch 1/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 245s 417ms/step - accuracy: 0.7616 - loss: 0.5594 - val_accuracy: 0.7583 - val_loss: 0.5406
Epoch 2/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 227s 407ms/step - accuracy: 0.7631 - loss: 0.5368 - val_accuracy: 0.7583 - val_loss: 0.5350
Epoch 3/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 247s 378ms/step - accuracy: 0.7636 - loss: 0.5330 - val_accuracy: 0.7657 - val_loss: 0.5264
Epoch 4/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 210s 377ms/step - accuracy: 0.7677 - loss: 0.5263 - val_accuracy: 0.7625 - val_loss: 0.5265
Epoch 5/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 208s 374ms/step - accuracy: 0.7708 - loss: 0.5206 - val_accuracy: 0.7698 - val_loss: 0.5207
Epoch 6/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 210s 376ms/step - accuracy: 0.7678 - loss: 0.5226 - val_accuracy: 0.7691 - val_loss: 0.5203
Epoch 7/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 262s 376ms/step - accuracy: 0.7697 - loss: 0.5179 - val_accuracy: 0.7684 - val_loss: 0.5124
Epoch 8/10
556/556 ━━━━━━━━━━━━━━━━━━━━ 211s 378ms/step - accuracy: 0.7743 -

In [21]:
# Predict the labels for the validation data
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int).flatten()

# Generate and print the classification report
report = classification_report(y_val, y_pred, target_names=['Not Hate Speech', 'Hate Speech'])
print(report)


139/139 ━━━━━━━━━━━━━━━━━━━━ 19s 134ms/step
                 precision    recall  f1-score   support

Not Hate Speech       0.78      0.98      0.87      3369
    Hate Speech       0.63      0.12      0.20      1074

       accuracy                           0.77      4443
      macro avg       0.70      0.55      0.53      4443
   weighted avg       0.74      0.77      0.70      4443



#### CNN performing better than LSTM